In [1]:
spark.stop()

NameError: name 'spark' is not defined

In [2]:
import os
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, lit
from pyspark.sql.types import BooleanType, StringType
from typing import List, Dict, Union
import concurrent.futures

# Step 1: Initialize Spark Session
# Set environment variables
os.environ['JAVA_HOME'] = '/Library/Java/JavaVirtualMachines/jdk-21.jdk/Contents/Home'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--master local pyspark-shell'

# Initialize findspark
findspark.init()
#spark.stop()
# Initialize Spark session
spark = SparkSession.builder \
    .appName("GenericDataGenerationExample") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.1.0") \
    .getOrCreate()



24/06/22 13:31:33 WARN Utils: Your hostname, Sthitaprajnas-MacBook-M1Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.61 instead (on interface en0)
24/06/22 13:31:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/pappsmac/.ivy2/cache
The jars for the packages stored in: /Users/pappsmac/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-136fa463-533d-401d-b513-b280bd375853;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Users/pappsmac/opt/anaconda3/envs/synthetic/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found io.delta#delta-core_2.12;2.1.0 in central
	found io.delta#delta-storage;2.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-core_2.12/2.1.0/delta-core_2.12-2.1.0.jar ...
	[SUCCESSFUL ] io.delta#delta-core_2.12;2.1.0!delta-core_2.12.jar (165ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-storage/2.1.0/delta-storage-2.1.0.jar ...
	[SUCCESSFUL ] io.delta#delta-storage;2.1.0!delta-storage.jar (34ms)
downloading https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.8/antlr4-runtime-4.8.jar ...
	[SUCCESSFUL ] org.antlr#antlr4-runtime;4.8!antlr4-runtime.jar (40ms)
downloading https://repo1.maven.org/maven2/org/codehaus/jackson/jackson-core-asl/1.9.13/jackson-core-asl-1.9.13.jar ...
	[SUCCESSFUL ] org.codehaus.jackson#jackson-core-asl;1.9.13!jackson-core-asl.jar (38ms)
:: resolution report :: resolve 907ms :: artifacts dl 283ms
	:: module

In [4]:
# Sample data
from pyspark.sql import SparkSession
from pyspark.sql.functions import rand, when, lit
import random



# Create a sample DataFrame
def create_sample_dataframe(num_rows=30):
    data = [(
        i,  # id
        random.randint(20, 60),  # age
        random.randint(30000, 100000),  # salary
        random.choice(['IT', 'HR', 'Finance', 'Marketing', 'Sales'])  # department
    ) for i in range(1, num_rows + 1)]

    df = spark.createDataFrame(data, ["id", "age", "salary", "department"])
    
    # Add some randomness to make the data more interesting
    df = df.withColumn("salary", when(rand() > 0.7, df.salary * 1.5).otherwise(df.salary))
    df = df.withColumn("age", when(rand() > 0.8, lit(18)).otherwise(df.age))

    return df

# Create the sample DataFrame
sample_df = create_sample_dataframe(30)

# Show the DataFrame
print("Sample DataFrame:")
sample_df.show()

# Save the DataFrame as a table
table_name = "employees2"
sample_df.write.mode("overwrite").saveAsTable(table_name)

print(f"DataFrame saved as table: {table_name}")

# To verify, you can read the table back
df_from_table = spark.table(table_name)
print("\nDataFrame read from table:")
df_from_table.show()




Sample DataFrame:
+---+---+--------+----------+
| id|age|  salary|department|
+---+---+--------+----------+
|  1| 33| 98833.5|        HR|
|  2| 33| 66562.5|        HR|
|  3| 53| 53133.0|        HR|
|  4| 50| 42029.0|     Sales|
|  5| 44| 32924.0| Marketing|
|  6| 59| 71259.0|   Finance|
|  7| 22| 35696.0|   Finance|
|  8| 18| 51879.0|     Sales|
|  9| 29| 62150.0|        HR|
| 10| 44| 64152.0|   Finance|
| 11| 50| 64304.0| Marketing|
| 12| 22| 43051.0|     Sales|
| 13| 39| 66033.0|     Sales|
| 14| 43| 41702.0|   Finance|
| 15| 18| 57894.0|   Finance|
| 16| 22| 62746.0|   Finance|
| 17| 56| 57478.0|   Finance|
| 18| 38| 77784.0|        IT|
| 19| 60| 59880.0| Marketing|
| 20| 52|132763.5| Marketing|
+---+---+--------+----------+
only showing top 20 rows

DataFrame saved as table: employees2

DataFrame read from table:
+---+---+--------+----------+
| id|age|  salary|department|
+---+---+--------+----------+
|  1| 33| 98833.5|        HR|
|  2| 33| 66562.5|        HR|
|  3| 53| 53133.0|   

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, BooleanType
import random
import string

def generate_random_string(length):
    return ''.join(random.choices(string.ascii_lowercase, k=length))

def generate_random_email():
    username = generate_random_string(8)
    domain = generate_random_string(5)
    return f"{username}@{domain}.com"

def generate_random_phone():
    return f"{random.randint(100, 999)}-{random.randint(100, 999)}-{random.randint(1000, 9999)}"

def generate_customer_data(num_rows):
    data = []
    for i in range(num_rows):
        customer = (
            i + 1000,  # customer_id
            generate_random_string(7),  # first_name
            generate_random_string(8),  # last_name
            generate_random_email(),  # email
            generate_random_phone(),  # phone_number
            random.randint(300, 850),  # credit_score
            round(random.uniform(0, 100000), 2),  # account_balance
            random.choice([True, False])  # is_active
        )
        data.append(customer)
    return data



# Define the schema for the customers table
schema = StructType([
    StructField("customer_id", IntegerType(), False),
    StructField("first_name", StringType(), False),
    StructField("last_name", StringType(), False),
    StructField("email", StringType(), False),
    StructField("phone_number", StringType(), False),
    StructField("credit_score", IntegerType(), False),
    StructField("account_balance", DoubleType(), False),
    StructField("is_active", BooleanType(), False)
])

# Generate the data
num_rows = 20
customer_data = generate_customer_data(num_rows)

# Create a DataFrame from the generated data
df_customers = spark.createDataFrame(customer_data, schema)

# Show the generated data
print("Generated Customer Data:")
df_customers.show()

# Save as a table
table_name = "customers"
df_customers.write.mode("overwrite").saveAsTable(table_name)
print(f"Data saved as table: {table_name}")

# Verify the saved data
print("\nVerifying saved data:")
spark.table(table_name).show()



Generated Customer Data:
+-----------+----------+---------+------------------+------------+------------+---------------+---------+
|customer_id|first_name|last_name|             email|phone_number|credit_score|account_balance|is_active|
+-----------+----------+---------+------------------+------------+------------+---------------+---------+
|       1000|   ddnfzrv| idpegens|owyjlvlr@ihwyq.com|868-796-1569|         783|       99999.64|    false|
|       1001|   hyjcdaj| imcxcrkb|negmrysm@zemrt.com|818-456-9245|         525|       69509.03|     true|
|       1002|   zgkqbsl| nnlsrcde|uroefoph@ptnlp.com|725-990-6281|         687|       75089.69|     true|
|       1003|   krztpaw| vwztywvh|fkgohydp@cjfdg.com|558-706-2373|         674|       91632.33|     true|
|       1004|   qfznnsf| eubamudu|zwjlcmwf@khzgo.com|131-425-4309|         454|        9779.89|    false|
|       1005|   kqlwccq| xztnoepz|nqhsrvhw@jnoay.com|674-846-3785|         497|       30723.59|     true|
|       1006|   dittn

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, lit, current_timestamp
from pyspark.sql.types import StructType, StructField, StringType, LongType, TimestampType
from typing import List, Dict
import concurrent.futures
from datetime import datetime

class Rule:
    def __init__(self, name: str, condition: str):
        self.name = name
        self.condition = condition

class RulesEngine:
    def __init__(self, spark: SparkSession):
        self.spark = spark
        self.metrics_schema = StructType([
            StructField("table_name", StringType(), False),
            StructField("rule_name", StringType(), False),
            StructField("passed", LongType(), False),
            StructField("failed", LongType(), False),
            StructField("total", LongType(), False),
            StructField("timestamp", TimestampType(), False)
        ])
        self.metrics_df = spark.createDataFrame([], schema=self.metrics_schema)

    def apply_rule(self, df, rule: Rule, table_name: str):
        result = df.withColumn(rule.name, expr(rule.condition))
        passed = result.filter(expr(f"{rule.name} == true")).count()
        failed = result.filter(expr(f"{rule.name} == false")).count()
        total = passed + failed

        metrics = {
            "table_name": table_name,
            "rule_name": rule.name,
            "passed": passed,
            "failed": failed,
            "total": total,
            "timestamp": datetime.now()
        }

        new_row = self.spark.createDataFrame([metrics], schema=self.metrics_schema)
        self.metrics_df = self.metrics_df.union(new_row)

        return metrics

    def run_rules(self, table_name: str, rules: List[Rule], filter_condition: str = None, num_partitions: int = None):
        df = self.spark.table(table_name)
        if filter_condition:
            df = df.filter(expr(filter_condition))
        if num_partitions:
            df = df.repartition(num_partitions)

        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = [executor.submit(self.apply_rule, df, rule, table_name) for rule in rules]
            results = [future.result() for future in concurrent.futures.as_completed(futures)]

        return self.spark.createDataFrame(results)

    def save_metrics(self, path: str):
        self.metrics_df.write.format("parquet").mode("append").save(path)

def run_rules_for_tables(spark: SparkSession, tables_config: List[Dict], metrics_path: str):
    engine = RulesEngine(spark)

    for config in tables_config:
        table_name = config['table_name']
        rules = [Rule(name=rule['name'], condition=rule['condition']) for rule in config['rules']]
        filter_condition = config.get('filter_condition')
        num_partitions = config.get('num_partitions')

        print(f"Running rules for table: {table_name}")
        results = engine.run_rules(table_name, rules, filter_condition, num_partitions)
        results.show()

    print("Saving metrics to Delta table")
    engine.save_metrics(metrics_path)






In [9]:
# Define rules
tables_config = [
    {
        "table_name": "employees2",
        "rules": [
            {"name": "age_check", "condition": "age >= 18"},
            {"name": "salary_check", "condition": "salary > 50000"},
            {"name": "department_check", "condition": "department == 'IT'"},
            {"name": "complex_check", "condition": "age > 30 and salary > 75000"}
        ],
        "filter_condition": "age > 25",
        "num_partitions": 2
    },
    {
        "table_name": "customers",
        "rules": [
            {"name": "credit_score_check", "condition": "credit_score > 700"},
            {"name": "balance_check", "condition": "account_balance > 1000"},
            {"name": "active_check", "condition": "is_active == true"}
        ],
        "num_partitions": 4
    }
    # Add more table configurations as needed
]

metrics_path = "./metrics_delta_table"

run_rules_for_tables(spark, tables_config, metrics_path)




Running rules for table: employees2
+------+------+----------------+----------+--------------------+-----+
|failed|passed|       rule_name|table_name|           timestamp|total|
+------+------+----------------+----------+--------------------+-----+
|     0|    22|       age_check|employees2|2024-06-22 13:34:...|   22|
|    19|     3|department_check|employees2|2024-06-22 13:34:...|   22|
|    18|     4|   complex_check|employees2|2024-06-22 13:34:...|   22|
|     5|    17|    salary_check|employees2|2024-06-22 13:34:...|   22|
+------+------+----------------+----------+--------------------+-----+

Running rules for table: customers
+------+------+------------------+----------+--------------------+-----+
|failed|passed|         rule_name|table_name|           timestamp|total|
+------+------+------------------+----------+--------------------+-----+
|     0|    20|     balance_check| customers|2024-06-22 13:34:...|   20|
|    16|     4|credit_score_check| customers|2024-06-22 13:34:...|  